# Setting up the model

In [2]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device


'cpu'

In [3]:
from utils.model import create_effnetb2_model
effnetb2, effnetb2_transforms  = create_effnetb2_model()

In [4]:
from torchvision import transforms
image_transform = transforms.Compose([transforms.TrivialAugmentWide(), effnetb2_transforms])

In [5]:
from torchinfo import summary
summary(effnetb2,
        input_size=(1, 3, 228, 228),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [1, 3, 228, 228]     [1, 101]             --                   Partial
├─Sequential (features)                                      [1, 3, 228, 228]     [1, 1408, 8, 8]      --                   False
│    └─Conv2dNormActivation (0)                              [1, 3, 228, 228]     [1, 32, 114, 114]    --                   False
│    │    └─Conv2d (0)                                       [1, 3, 228, 228]     [1, 32, 114, 114]    (864)                False
│    │    └─BatchNorm2d (1)                                  [1, 32, 114, 114]    [1, 32, 114, 114]    (64)                 False
│    │    └─SiLU (2)                                         [1, 32, 114, 114]    [1, 32, 114, 114]    --                   --
│    └─Sequential (1)                                        [1, 32, 114, 114]    [1, 1

# Getting Data

In [6]:
from torchvision import datasets
from pathlib import Path
data_dir = Path('data')
train_data = datasets.Food101(root = data_dir,
                              split = "train",
                              transform=image_transform,
                              download = True)
test_data = datasets.Food101(root=data_dir,
                             split = "test",
                             transform = effnetb2_transforms,
                             download = True)

Using downloaded and verified file: data\food-101.tar.gz
Extracting data\food-101.tar.gz to data


In [7]:
from torch.utils.data import random_split
import torchvision
def split_datasets(dataset:torchvision.datasets,
                  split_size = 0.2,
                  seed = 42):
    #Create split length
    length_1 = int(len(dataset)*split_size)
    length_2 = len(dataset) - length_1
    random_split_1, random_split_2 = torch.utils.data.random_split(dataset,
                                                                   [length_1,length_2],
                                                                  generator = torch.manual_seed(seed))
    return random_split_1, random_split_2

In [8]:
train_data_food101_40_percent, _ = split_datasets(dataset = train_data,
                                                 split_size = 0.4)
test_data_food101_40_percent, _ = split_datasets(dataset = test_data,
                                                 split_size = 0.4)

In [9]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(
                              dataset = train_data_food101_40_percent,                          
                              batch_size= 32,
                              shuffle = True,
                              )
test_dataloader =  DataLoader(
                              dataset=test_data_food101_40_percent,
                              batch_size= 32,
                              shuffle = False,
                              )

# Training the model

In [10]:
import torch
from utils import engine

loss_fn = torch.nn.CrossEntropyLoss(label_smoothing = 0.1)
optimizer = torch.optim.Adam(params = effnetb2.parameters(), lr=5e-4, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,"min",patience=5)
early_stopping = engine.EarlyStopping(tolerance=5, min_delta=10)

In [14]:
from utils import engine
EPOCHS = 1

results = engine.train(model=effnetb2,
                      train_dataloader=train_dataloader,
                      test_dataloader=test_dataloader,
                      epochs=EPOCHS,
                      optimizer=optimizer,
                      loss_fn=loss_fn,
                      scheduler = scheduler,
                      early_stopping = early_stopping,
                      device=device)

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from utils.util import plot_loss_curves

plot_loss_curves(results)

# Saving Model

In [ ]:
from utils.util import save_model
save_model(model = effnetb2,
                   target_dir = "models",
                   model_name = f"mobileNetV3_quickdraw_animals_epoch_{EPOCHS}.pth")